# HER + multi agent ( PPO + DDPG ) cooperation + multitask ( 3D navigation )
- Reacher 20 Arms ( UnityML ) ~ experimental Work In Progress ( well can takes a time to make this )
- not working, concept of idea i am heading to check 
- however before i need properly benchmark reacher with my framework ( w/o HER or fancy stuffs )
- here is just for demonstrating idea + implementation ~ so framework is OK to run it

### Import generics

In [ ]:
import os, time

os.chdir("..")

import numpy as np
import toml, gym

import torch
from torch.multiprocessing import Queue, Process

### Prepare statistic helpers

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def print_stats(scores, title):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, len(scores)+1), scores)
    plt.ylabel('Rewards')
    plt.xlabel(title)
    plt.show()

### Load task configs ~ this should be adopted offline for particular task

In [ ]:
CFG = toml.loads(open('cfg.toml').read())
DDPG_CFG = toml.loads(open('experimental/ddpg_cfg.toml').read())
PPO_CFG = toml.loads(open('experimental/ppo_cfg.toml').read())

torch.set_default_tensor_type(CFG['tensor'])
print(CFG['task'])
CFG['task'] = "data/Reacher.x86_64"
CFG['total_simulations'] = 20

### Import wheeler environment and particular utils we want to use ~ general ones ( shared across tasks )

In [ ]:
from utils.task import Task
from utils.taskinfo import *

from utils.rbf import *
from utils.normalizer import *

from utils.taskmgr import *
from utils.replay import *

from utils.fastmem import Memory

from utils.curiosity import *

from utils.unity import unity_factory
from utils.crossexp import *

from agent.zer0bot import agent_launch

### Define Task wrapper ~ when is goal met, how to step ( update rewards function, .. ), when / how to reset

In [ ]:
CLOSE_ENOUGH = 1.25

def extract_goal(state):
    return state[-4-3:-1-3] # this is nice, from perspective, that we are OK if our states are stacked ~ we work with latest one

def goal_distance(goal_a, goal_b):
    return np.linalg.norm(goal_a - goal_b.reshape(goal_a.shape), axis=-1)

def fun_reward(s, n, goal, objective_id, cfg, her):
    xid = objective_id % 4
    if xid >= 3: # full task 3D naigation #True:#
        return -1 * (2 * CLOSE_ENOUGH < np.abs(goal_distance(extract_goal(n), goal)))
# X- Y - Z subtask navigation ~ multi task
    a = np.abs(extract_goal(n)[xid] - goal[xid])
    b = np.abs(extract_goal(s)[xid] - goal[xid])
    if b < CLOSE_ENOUGH:
        return 0.
    return -1 + .9 * int(a < b) # little shape it maybe good idea ? ~ experimental

def sample_goal(cfg, trajectory_goal, trajectory_state):
    hs = cfg['her_state_size']
    pos = extract_goal(trajectory_state.copy())
    for i in range(3):# be carefull extremly expensive
        radius = np.abs(np.random.rand() * CLOSE_ENOUGH)
        angle = np.random.rand() * np.pi * 2
        a = np.cos(angle) * radius
        b = np.sin(angle) * radius
        ids = np.random.choice(hs, 2, p=[1/hs]*hs, replace=False)

        target = pos if random.randint(0, 5) else trajectory_goal
        goal = target.copy()
        goal[ids[0]] += a
        goal[ids[1]] += b

        if np.abs(goal_distance(goal, target)) < CLOSE_ENOUGH:
            return goal, target
    return (
        pos if random.randint(0, 3) else trajectory_goal, 
        pos if random.randint(0, 3) else trajectory_goal
    )

def goal_select(total_after, n_step):
    if total_after <= n_step + 1: # only last n_state remainds
        return 0
    if random.randint(0, 2):
        return random.randint(1, n_step)
    if 0 == random.randint(0, 3):
        return 0
    return random.randint(1, total_after - 1 - n_step)

In [ ]:
class GymTask(Task):
    def reset(self, seed = None, test = False):
        cfg = {"goal_size":5., "goal_speed":0.}
        state = super().reset(cfg, test)[0]

        self.goal_ = state
        
        if test: return state # we will get array of states
        
        state = state.reshape(-1)
        self.prev = state
        return [ state ]
    
    def step_ex(self, action, test = False):
        state, done, reward = self.env.step(self.bot_id, self.objective_id, action)
        self.goal_ = state.copy()

        if not test:
            reward = fun_reward(
                self.prev.reshape(state.shape), state,
                self.goal(0).reshape(-1), self.objective_id, 
                self.cfg, False)
        self.prev = state
        return action, state, reward, done, True

    def goal_met(self, states, rewards, n_steps):
        return sum(abs(r) for r in rewards) > 30

    def goal(self, ind = 0):
#        print(extract_goal(self.goal_.reshape(-1)).reshape(1, -1) if self.goal_.shape[0] != 20 else extract_goal(self.goal_[ind].reshape(-1)).reshape(1, -1))
        if self.goal_.shape[0] != 20: return extract_goal(self.goal_.reshape(-1)).reshape(1, -1)
        return extract_goal(self.goal_[ind].reshape(-1)).reshape(1, -1)
    
    def update_goal(self, _, goals, states, n_goals, n_states, updates):
        for i, (g, s, n_g, n, u) in enumerate(zip(goals, states, n_goals, n_states, updates)):
            if u:
                ind = i + goal_select(len(states) - i, self.cfg['n_step'])
                g, n_g = sample_goal(self.cfg, goals[ind], states[ind])
            
            yield (
                fun_reward(s, n, g, self.objective_id, self.cfg, True),
                g, s,
                n_g, n,
            )

### Generic proxy for creating our Task ( multiprocess environments purpose mainly ) 
- but can also add wrapping function approx values ( action value to tanh, sigmoid, .. ) - this not works well with PPO now

In [ ]:
class GymInfo(TaskInfo):
    def __init__(self, replaybuf, factory, Mgr, args):
        super().__init__(
                33, 4, -1, +1,
                CFG,
                replaybuf,
                factory, Mgr, args)

    def new(self, cfg, bot_id, objective_id):
        # first task created must be our testing one ( not for learning, just for testing .. TODO : move inside framework )
        task = GymTask(cfg,
                self.env,
                objective_id, bot_id,
                self.action_low, self.action_high)
        if -1 == objective_id:
            task.reset()
        return task

### Implement callback for testing policy ~ per X training rounds, we want to test it ~ enable visuals if you want

In [ ]:
def context_callback(count, print_every):
    context_callback.z = 0
    context_callback.scores = []
    context_callback.total_scores = []
    def callback(task, agent, scores):        
        context_callback.z += 1
        context_callback.scores += scores
        if 0 == context_callback.z % print_every:
            print("\nTEST review : ", np.mean(context_callback.scores[-print_every:]), "LAST stats :", sum(scores), sum(map(lambda r: r != 0, scores)), len(scores))
        # we can save scores to main queue, and avarage them, or we can ..
        # run testing w/ visuals :
        for _ in range(count):
            done, states, rewards = task.test_policy(agent)
            rewards = sum(rewards)
            context_callback.total_scores.append(rewards)
            if not done:
                return None
    
        print_stats(context_callback.scores, "all-out::testing_phase")

        print("\n")
        print("="*80)
        print("training over", context_callback.z * DDPG_CFG['n_simulations'] * DDPG_CFG['mcts_rounds'])
        print("="*80)

        return context_callback.total_scores
    return callback

def ppo_callback(task, agent, scores): # agent cooperation #1 loading core of the actor model, and do sample efficient PPO on it
    agent.bot.alpha_sync(["neslayer_2_"] + ["_%i."%i for i in range(PPO_CFG["n_simulations"], DDPG_CFG["n_simulations"])])
    # we avoid loading last layer of NoisyNetwork ~ to let degree of freedom and adaptation
    # also explorer have less ( maybe should have more ? ) detached actors to work with, but we anyway sync it by actor so does not matter
    return None

### Prepare neural network which we will be using

In [ ]:
from models import ddpg_model, noisy_model, state_action_model

def CriticNN(state_size, action_size, wrap_value, cfg):
    return state_action_model.Critic(state_size, action_size, wrap_value, cfg, fcs1_units=256, fc2_units=128)

def ActorNN(state_size, action_size, wrap_action, cfg):
    return noisy_model.Actor(state_size, action_size, wrap_action, cfg, hiddens=[400, 300])
    return noisy_model.Actor(state_size, action_size, wrap_action, cfg, hiddens=[128, 64])

### Select encoders

In [ ]:
from utils.encoders import *
from utils.rnn import *#GRUEncoder

def encoderstack():
    norm = GlobalNormalizer(DDPG_CFG, 33)
    return norm # well GRU/LSTM testing comes if we go sucessfully at least here ...
    DDPG_CFG['max_n_episode'] = 70 # we want gae + rnn; need re-eval eps ~ 1000 per shot is overkill
    DDPG_CFG['batch_size'] = 16 # this is basically numero of sample eps, an from each ep we draw about <1, 40> steps
    experience = FasterGRUEncoder(DDPG_CFG, norm.total_size())#GRU#LSTM
    encoder_norm = StackedEncoder(DDPG_CFG, 33, norm, experience)
    encoder_norm.share_memory()
    return encoder_norm

### Cook Task : replay buffer ( fast / prio-gae-rnn ) + task manager ( local / remote / unity )

In [ ]:
def taskfactory(): # agent cooperation #2 sharing experiences
    return GymInfo(cross_exp_buffer(CFG), unity_factory(CFG, CFG['total_simulations']), RemoteTaskManager, (LocalTaskManager, 1 + CFG['total_simulations']))

### Glue it all together ~ select buffer, encoders, agents, ... and RUN!!

In [ ]:
class GoalEncoder(IEncoder):
    def __init__(self, cfg, size):
        super().__init__(cfg)
        self.size = size
        self.net = torch.nn.Linear(size, cfg['her_state_features'], bias=False)
    def out_size(self):
        assert False, "..."
    def forward(self, goals, _):
        return self.net(goals)

def main():
    print(CFG)
    
    encoder = encoderstack()
    task_factory = taskfactory()
    task = task_factory.new(DDPG_CFG, 0, -1)
    
    callback = context_callback(10, 10)
    def callback_task(agent, stop_q):
        return callback(task, agent, stop_q)
    
    def ppo_callback_wrap(agent, stop_q):
        return ppo_callback(task, agent, stop_q)

    stop_q = Queue()
    
    #import threading
    worker = Process(#threading.Thread(#
        target=agent_launch,
        args=(
            1,
            PPO_CFG, task_factory, 
            encoder, ActorNN, CriticNN,
            stop_q, ppo_callback_wrap,
            GoalEncoder(PPO_CFG, PPO_CFG['her_state_size']),))

    worker.start() # agent for efficient explorering!
    # now agent for complex learning ( efficiently handle pas experiences )
    agent_launch(0, DDPG_CFG, task_factory, encoder, ActorNN, CriticNN, stop_q, callback_task, GoalEncoder(DDPG_CFG, DDPG_CFG['her_state_size']))
    
    scores = stop_q.get()
    print("FINISHED!")
    print_stats(scores, "learning algorithm")
    
    worker.join()

if '__main__' == __name__:
    main()